In [1]:
### Developing a real stratergy to invest in the stock market

In [ ]:
### When developing our stratergy, the goal will be to acquire good and useful information about each 
# Stock. We then need to check which parameters play a role in explaining variation in the data and
# which simply contribute noise to the system. Once we have generated and calculated the financial indicators
# and then optimised the tuning of these params using something like simulated annealing to search
# the complex solution space we can test and compare our stratergy against the S&P500 real life data. 

# Sentiment data could be useful, financial statements even better, unsure how to extract, useful
# information from these datasets.

# Looking for monthly readjustments for our stock portfolio. Further investigation should be carried 
# on arbitrage. 

In [1]:
import yfinance as yf
import pandas as pd
import os

In [17]:
# def download_stock_data(ticker, start_date, end_date):
#     stock_data = yf.download(ticker, start_date, end_date)
#     return stock_data


# ticker = 'AAPL'
# start_date = "2020-01-01"
# end_date = "2023-12-12"

# data = download_stock_data(ticker, start_date, end_date)

# data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

data = pd.read_csv("C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics/AAPL_stock_data.csv")

data.head()

# data.to_csv('AAPL_stock_data.csv', index=True)

,Price,Adj Close,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.79600524902344,75.0875015258789,75.1500015258789,73.79750061035156,74.05999755859375,135480400
3,2020-01-03,72.08830261230469,74.35749816894531,75.1449966430664,74.125,74.2874984741211,146322800
4,2020-01-06,72.6627197265625,74.94999694824219,74.98999786376953,73.1875,73.44750213623047,118387200


In [46]:
# data = data.iloc[2:]

#data.rename(columns={data.columns[0]: 'Date'}, inplace=True)

# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# data

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-01-02,72.79600524902344,75.0875015258789,75.1500015258789,73.79750061035156,74.05999755859375,135480400
2020-01-03,72.08830261230469,74.35749816894531,75.1449966430664,74.125,74.2874984741211,146322800
2020-01-06,72.6627197265625,74.94999694824219,74.98999786376953,73.1875,73.44750213623047,118387200
2020-01-07,72.32095336914062,74.59750366210938,75.2249984741211,74.37000274658203,74.95999908447266,108872000
2020-01-08,73.48432159423828,75.79750061035156,76.11000061035156,74.29000091552734,74.29000091552734,132079200
...,...,...,...,...,...,...
2023-12-05,192.47705078125,193.4199981689453,194.39999389648438,190.17999267578125,190.2100067138672,66628400
2023-12-06,191.38241577148438,192.32000732421875,194.75999450683594,192.11000061035156,194.4499969482422,41089700
2023-12-07,193.32290649414062,194.27000427246094,195.0,193.58999633789062,193.6300048828125,47477700


In [2]:
### Define and calcuate RSI 
def calculate_rsi(data, column, window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    # Calculate daily price changes (delta)
    delta = data[column].diff()

    # Separate gains and losses
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    
    avg_gain = gains.rolling(window=window, min_periods=window).mean()
    avg_loss = losses.rolling(window=window, min_periods=window).mean()

    rs = avg_gain/avg_loss

    rsi = 100 - (100/ (1+rs))
    
    return rsi

def calculate_rsi_ewma(data, column="Adj Close", window=14):

    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data must be a pandas DataFrame")

    if column not in data.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame")

    # Calculate daily price changes (delta)
    delta = data[column].diff()

    # Separate gains and losses
    gains = delta.clip(lower=0)
    losses = -delta.clip(upper=0)
    
    avg_gain = gains.ewm(span=window, adjust=False).mean()
    avg_loss = losses.ewm(span=window, adjust=False).mean()

    rs = avg_gain/avg_loss

    rsi = 100 - (100/ (1+rs))
    
    return rsi

In [3]:
column = "Adj Close"

data['RSI'] = calculate_rsi(data=data, column=column, window=30)

data['RSI_ewma'] = calculate_rsi_ewma(data=data, column=column, window=14)

data

NameError: name 'data' is not defined

In [24]:
data

Price,Adj Close,Close,High,Low,Open,Volume,RSI,RSI_ewma
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,,
Date,,,,,,,,
2020-01-02,72.796005,75.087502,75.150002,73.797501,74.059998,135480400,NaN,NaN
2020-01-03,72.088303,74.357498,75.144997,74.125000,74.287498,146322800,NaN,0.000000
2020-01-06,72.662720,74.949997,74.989998,73.187500,73.447502,118387200,NaN,11.100954
2020-01-07,72.320953,74.597504,75.224998,74.370003,74.959999,108872000,NaN,10.314862
2020-01-08,73.484322,75.797501,76.110001,74.290001,74.290001,132079200,NaN,29.831016
...,...,...,...,...,...,...,...,...
2023-12-05,192.477051,193.419998,194.399994,190.179993,190.210007,66628400,NaN,70.647406
2023-12-06,191.382416,192.320007,194.759995,192.110001,194.449997,41089700,NaN,63.738579
